In [4]:
import tensorflow as tf
#import tensorflow_hub as hub
import requests
print("TensorFlow version: {}".format(tf.__version__))
import zipfile
import requests
import os
import time
import pandas as pd
import random
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, AvgPool2D, Dense, Concatenate, Flatten, Lambda, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.losses import CategoricalCrossentropy
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, CSVLogger
import numpy as np
from PIL import Image
import tensorflow.keras.backend as K
import pickle
from tensorflow.keras.models import load_model, Model

TensorFlow version: 2.17.0


In [5]:
gpus = tf.config.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except:
        print("Couldn't set memory_growth")
        pass
    
    
def fix_random_seed(seed):
    """ Setting the random seed of various libraries """
    try:
        np.random.seed(seed)
    except NameError:
        print("Warning: Numpy is not imported. Setting the seed for Numpy failed.")
    try:
        tf.random.set_seed(seed)
    except NameError:
        print("Warning: TensorFlow is not imported. Setting the seed for TensorFlow failed.")
    try:
        random.seed(seed)
    except NameError:
        print("Warning: random module is not imported. Setting the seed for random failed.")

# Fixing the random seed
random_seed=4321
fix_random_seed(random_seed)

[]


In [6]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

['/device:CPU:0']


In [7]:
import os
import requests
import zipfile

# Retrieve the data
if not os.path.exists(os.path.join('data','tiny-imagenet-200.zip')):
    url = "http://cs231n.stanford.edu/tiny-imagenet-200.zip"
    # Get the file from web
    r = requests.get(url)

    if not os.path.exists('data'):
        os.mkdir('data')
    
    # Write to a file
    with open(os.path.join('data','tiny-imagenet-200.zip'), 'wb') as f:
        f.write(r.content)
else:
    print("The zip file already exists.")
    
if not os.path.exists(os.path.join('data', 'tiny-imagenet-200')):
    with zipfile.ZipFile(os.path.join('data','tiny-imagenet-200.zip'), 'r') as zip_ref:
        zip_ref.extractall('data')
else:
    print("The extracted data already exists")

The zip file already exists.
The extracted data already exists


In [8]:
from functools import partial
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def get_test_labels(tpath):
    test_df  = pd.read_csv(tpath , sep = '\t' , header = None , index_col= None)
    test_df = test_df.iloc[:,[0,1]].rename({0:'filename' , 1: 'class'} , axis = 1)
    return test_df

def get_datagen(target_size , batch_size):
    # print(target_size)
    image_gen_aug = ImageDataGenerator(samplewise_center=False, rotation_range=30, width_shift_range=0.2,
            height_shift_range=0.2, brightness_range=(0.5,1.5), shear_range=5, 
            zoom_range=0.2, horizontal_flip=True, fill_mode='reflect', validation_split=0.1)
    image_gen = ImageDataGenerator(samplewise_center=False)

    partial_flow = partial(image_gen_aug.flow_from_directory , directory = os.path.join('data','tiny-imagenet-200','train'),
                           classes = None , class_mode = 'categorical' , batch_size = batch_size , target_size = target_size , shuffle = True , seed = random_seed)
    train_gen = partial_flow(subset = 'training')
    val_gen = partial_flow(subset = 'validation')

    test_df = get_test_labels(os.path.join('data','tiny-imagenet-200', 'val','val_annotations.txt'))
    test_gen = image_gen.flow_from_dataframe(test_df, directory=os.path.join('data','tiny-imagenet-200',  'val', 'images'), target_size=target_size, classes=None,
        class_mode='categorical', batch_size=batch_size, shuffle=False)
    
    return train_gen , val_gen , test_gen

def data_gen_augmented_inceptionnet_v1(gen, random_gamma=False, random_occlude=False):
    for x,y in gen: 
        
        if x.ndim != 4:
            raise ValueError("This function is designed for a batch of images with 4 dims [b, h, w, c]")
            
        if random_gamma:
            # Gamma correction
            # Doing this in the image process fn doesn't help improve performance
            rand_gamma = np.random.uniform(0.9, 1.08, (x.shape[0],1,1,1))
            x = x**rand_gamma
        
        if random_occlude:
            # Randomly occluding sections in the image
            occ_size = 10
            occ_h, occ_w = np.random.randint(0, x.shape[1]-occ_size), np.random.randint(0, x.shape[2]-occ_size)
            x[::2,occ_h:occ_h+occ_size,occ_w:occ_w+occ_size,:] = np.random.choice([0.,128.,255.])
        
        # Image centering
        x -= np.mean(x, axis=(1,2,3), keepdims=True)
        
        # Making sure we replicate the target (y) three times
        yield x,(y,y,y)


batch_size = 128
target_size = (56,56)
# print(target_size, 'outside')
train_gen, valid_gen, test_gen = get_datagen(target_size, batch_size)
# Modifying the data generators to fit the model targets
# We augment data in the training set
train_gen_aux = data_gen_augmented_inceptionnet_v1(train_gen, random_gamma=True, random_occlude=True)
# We do not augment data in the validation/test datasets
valid_gen_aux = data_gen_augmented_inceptionnet_v1(valid_gen)
test_gen_aux = data_gen_augmented_inceptionnet_v1(test_gen)


Found 90000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.
Found 10000 validated image filenames belonging to 200 classes.


In [13]:
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, AvgPool2D, Dense, Concatenate, Flatten, Lambda, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.losses import CategoricalCrossentropy
import tensorflow.keras.backend as K

def inception(inp, n_filters):

    # 1x1 layer
    # init argument defaults to glorot_uniform
    out1 = Conv2D(n_filters[0][0], (1,1), strides=(1,1), activation='relu', padding='same')(inp)

    # 1x1 followed by 3x3
    out2_1 = Conv2D(n_filters[1][0], (1,1), strides=(1,1), activation='relu', padding='same')(inp)
    out2_2 = Conv2D(n_filters[1][1], (3,3), strides=(1,1), activation='relu', padding='same')(out2_1)

    # 1x1 followed by 5x5
    out3_1 = Conv2D(n_filters[2][0], (1,1), strides=(1,1), activation='relu', padding='same')(inp)
    out3_2 = Conv2D(n_filters[2][1], (5,5), strides=(1,1), activation='relu', padding='same')(out3_1)

    # 3x3 (pool) followed by 1x1
    out4_1 = MaxPool2D((3,3), strides=(1,1), padding='same')(inp)
    out4_2 = Conv2D(n_filters[3][0], (1,1), strides=(1,1), activation='relu', padding='same')(out4_1)

    out = Concatenate(axis=-1)([out1, out2_2, out3_2, out4_2])
    return out

# Code listing 7.4
def aux_out(inp,name=None):    
    avgpool1 = AvgPool2D((5,5), strides=(3,3), padding='valid')(inp)
    conv1 = Conv2D(128, (1,1), activation='relu', padding='same')(avgpool1)
    flat = Flatten()(conv1)
    dense1 = Dense(1024, activation='relu')(flat) 
    dense1 = Dropout(0.7)(dense1)
    aux_out = Dense(200, activation='softmax', name=name)(dense1)
    return aux_out

def stem(inp):
    conv1 = Conv2D(64, (7,7), strides=(1,1), activation='relu', padding='same')(inp)
    maxpool2 = MaxPool2D((3,3), strides=(2,2), padding='same')(conv1)
    lrn3 = Lambda(lambda x: tf.nn.local_response_normalization(x))(maxpool2)

    conv4 = Conv2D(64, (1,1), strides=(1,1), padding='same')(lrn3)
    conv5 = Conv2D(192, (3,3), strides=(1,1), activation='relu', padding='same')(conv4)
    lrn6 = Lambda(lambda x: tf.nn.local_response_normalization(x))(conv5)

    maxpool7 = MaxPool2D((3,3), strides=(1,1), padding='same')(lrn6)

    return maxpool7


def inception_v1():
    
    K.clear_session()
    
    inp = Input(shape=(56,56,3))
    stem_out = stem(inp)
    inc_3a = inception(stem_out, [(64,),(96,128),(16,32),(32,)])
    inc_3b = inception(inc_3a, [(128,),(128,192),(32,96),(64,)])

    maxpool = MaxPool2D((3,3), strides=(2,2), padding='same')(inc_3b)

    inc_4a = inception(maxpool, [(192,),(96,208),(16,48),(64,)])
    inc_4b = inception(inc_4a, [(160,),(112,224),(24,64),(64,)])

    aux_out1 = aux_out(inc_4a, name='aux1')

    inc_4c = inception(inc_4b, [(128,),(128,256),(24,64),(64,)])
    inc_4d = inception(inc_4c, [(112,),(144,288),(32,64),(64,)])
    inc_4e = inception(inc_4d, [(256,),(160,320),(32,128),(128,)])
    
    maxpool = MaxPool2D((3,3), strides=(2,2), padding='same')(inc_4e)
    
    aux_out2 = aux_out(inc_4d, name='aux2')

    inc_5a = inception(maxpool, [(256,),(160,320),(32,128),(128,)])
    inc_5b = inception(inc_5a, [(384,),(192,384),(48,128),(128,)])
    avgpool1 = AvgPool2D((7,7), strides=(1,1), padding='valid')(inc_5b)

    flat_out = Flatten()(avgpool1)
    flat_out = Dropout(0.4)(flat_out)
    out_main = Dense(200, activation='softmax', name='final')(flat_out)

    model = Model(inputs=inp, outputs=[out_main, aux_out1, aux_out2])
    model.compile(loss='categorical_crossentropy', 
                       optimizer='adam', metrics=['accuracy' , 'accuracy' , 'accuracy'])
    return model

model = inception_v1()
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 56, 56, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 56, 56,    │      9,472 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 28, 28,    │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 28, 28,    │          0 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 28, 28,    │      4,160 │ lambda[0][0]      │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 28, 28,    │    110,784 │ conv2d_1[0][0]    │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, 28, 28,    │          0 │ conv2d_2[0][0]    │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 28, 28,    │          0 │ lambda_1[0][0]    │
│ (MaxPooling2D)      │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 28, 28,    │     18,528 │ max_pooling2d_1[… │
│                     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 28, 28,    │      3,088 │ max_pooling2d_1[… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 28, 28,    │          0 │ max_pooling2d_1[… │
│ (MaxPooling2D)      │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 28, 28,    │     12,352 │ max_pooling2d_1[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 28, 28,    │    110,720 │ conv2d_4[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 28, 28,    │     12,832 │ conv2d_6[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 28, 28,    │      6,176 │ max_pooling2d_2[… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 28, 28,    │          0 │ conv2d_3[0][0],   │
│ (Concatenate)       │ 256)              │            │ conv2d_5[0][0],   │
│                     │                   │            │ conv2d_7[0][0],   │
│                     │                   │            │ conv2d_8[0][0]  

 Total params: 10,918,280 (41.65 MB)

 Trainable params: 10,918,280 (41.65 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
def get_steps_per_epoch(n_data, batch_size):
    """ Given the data size and batch size, gives the number of steps to travers the full dataset """
    if n_data%batch_size==0:
        return int(n_data/batch_size)
    else:
        return int(n_data*1.0/batch_size)+1

In [16]:
if not os.path.exists('eval'):
    os.mkdir('eval')
    
# Early stopping callback
es_callback = EarlyStopping(monitor='val_loss', patience=5)
# Logging the performance metrics to a CSV file
csv_logger = CSVLogger(os.path.join('eval','2_eval_data_aug_early_stopping.log'))

t1 = time.time()
history = model.fit(
    train_gen_aux, validation_data=valid_gen_aux, 
    steps_per_epoch=get_steps_per_epoch(int(0.9*(500*200)), batch_size), 
    validation_steps=get_steps_per_epoch(int(0.1*(500*200)),batch_size),
    epochs=5, callbacks=[es_callback, csv_logger]
)
t2 = time.time()

print("It took {} seconds to complete the training".format(t2-t1))

Epoch 1/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 1664s 2s/step - aux1_accuracy: 0.0111 - aux2_accuracy: 0.0093 - final_accuracy: 0.0083 - loss: 15.5891 - val_aux1_accuracy: 0.0286 - val_aux2_accuracy: 0.0221 - val_final_accuracy: 0.0224 - val_loss: 14.9117
Epoch 2/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 1652s 2s/step - aux1_accuracy: 0.0339 - aux2_accuracy: 0.0263 - final_accuracy: 0.0287 - loss: 14.7341 - val_aux1_accuracy: 0.0657 - val_aux2_accuracy: 0.0544 - val_final_accuracy: 0.0529 - val_loss: 13.9506
Epoch 3/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 1628s 2s/step - aux1_accuracy: 0.0583 - aux2_accuracy: 0.0466 - final_accuracy: 0.0535 - loss: 13.9621 - val_aux1_accuracy: 0.0905 - val_aux2_accuracy: 0.0747 - val_final_accuracy: 0.0750 - val_loss: 13.3852
Epoch 4/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 1606s 2s/step - aux1_accuracy: 0.0819 - aux2_accuracy: 0.0648 - final_accuracy: 0.0732 - loss: 13.4017 - val_aux1_accuracy: 0.1182 - val_aux2_accuracy: 0.1031 - val_final_accuracy: 0.1057 - val_loss: 12.6938
Epoch 5/5
70

'data/tiny-imagenet-200'